# 초기 코드

### mark 1

In [2]:
import requests
from bs4 import BeautifulSoup
import newspaper
import time
import difflib
from datetime import datetime, timezone, timedelta
from urllib.parse import urljoin
from config import TARGET_SITES, CHECK_INTERVAL_SECONDS, MONGO_URI, MONGO_DB_NAME, MONGO_COLLECTION_NAME
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse, urlunparse
import re

# 설정
KST = timezone(timedelta(hours=9))
SCRIPT_START_TIME = datetime.utcnow().replace(tzinfo=timezone.utc).astimezone(KST)
#캐시 선언
recent_titles = deque(maxlen=100)
processed_urls = deque(maxlen=100)


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

last_seen_urls = {
    f"{site['name']} - {site.get('category', '')}": None for site in TARGET_SITES
}

client = MongoClient(MONGO_URI)
db = client[MONGO_DB_NAME]
articles_collection = db[MONGO_COLLECTION_NAME]

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def parse_chosunbiz_date(span_text):
    pattern = r'입력\s*(\d{4})\.(\d{2})\.(\d{2})\.?\s*(\d{2}):(\d{2})'
    match = re.search(pattern, span_text.strip())
    if match:
        year, month, day, hour, minute = match.groups()
        return datetime.strptime(f"{year}-{month}-{day} {hour}:{minute}", "%Y-%m-%d %H:%M")
    return None
import difflib
from collections import deque

# 최근 기사 제목 캐시
recent_titles = deque(maxlen=100)

def is_similar_title(new_title, threshold=0.9):
    for past_title in recent_titles:
        similarity = difflib.SequenceMatcher(None, new_title, past_title).ratio()
        if similarity >= threshold:
            print(f"⚠️ 제목 유사도 중복 판단 →")
            print(f"    ▸ 새로운 제목: {new_title}")
            print(f"    ▸ 유사한 제목: {past_title}")
            print(f"    ▸ 유사도: {similarity:.3f}")
            return True
    return False

def localize_to_kst(dt):
    if dt is None:
        return None
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc).astimezone(KST)
    return dt.astimezone(KST)

def clean_url(url):
    parsed = urlparse(url)
    return urlunparse(parsed._replace(query=""))

def fetch_article_details(url, site=None, driver=None):
    try:
        url = clean_url(url)  # ✨ URL 정리
        article = newspaper.Article(url, language='ko')
        article.download()
        article.parse()
        publish_time = article.publish_date

        if site and site['name'] == '조선비즈' and driver:
            driver.get(url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.inputDate")))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            span = soup.select_one("span.inputDate")
            if span:
                parsed = parse_chosunbiz_date(span.text)
                if parsed:
                    publish_time = parsed

        publish_time = localize_to_kst(publish_time)

        return {
            'title': article.title,
            'text': article.text,
            'publish_time': publish_time,
            'url': url
        }

    except Exception as e:
        print(f"[ERROR] 기사 처리 중 오류: {url}\n  └ {e}")
        return None

def save_article_to_mongodb(article_data):
    try:
        articles_collection.insert_one(article_data)
        print(f"  [DB 저장] ({article_data['publisher']}) {article_data['title']}")
    except Exception as e:
        print(f"[ERROR] DB 저장 실패: {e}")

def get_latest_articles(site, driver=None):
    try:
        if not site.get('selector'):
            print(f"[ERROR] {site['name']} - {site.get('category', '')} → selector 누락")
            return []

        if site.get('dynamic'):
            if driver:
                driver.get(site['url'])
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, site['selector']))
                )
                soup = BeautifulSoup(driver.page_source, 'html.parser')
            else:
                return []
        else:
            res = requests.get(site['url'], headers=headers, timeout=10)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, 'html.parser')

        # ✅ 컨테이너 영역 안에서만 기사 추출
        if site.get("container_selector"):
            container = soup.select_one(site["container_selector"])
            if not container:
                print(f"[ERROR] {site['name']} → container_selector 영역을 찾을 수 없습니다.")
                return []
            links = container.select(site["selector"])
        else:
            links = soup.select(site["selector"])

        articles = []
        for a in links:
            href = a.get('href')
            title = a.get_text(strip=True)
            if not href or site['pattern'] not in href:
                continue
            full_url = urljoin(site['base_url'], href)
            articles.append({'title': title, 'url': full_url})

        return articles
    except Exception as e:
        print(f"[ERROR] 목록 수집 실패 ({site['name']}) → {e}")
        return []


def monitor_news(driver=None, static_only=False):
    print("\n[실시간 뉴스 모니터링 시작]")
    print(f"  기준 시간: {SCRIPT_START_TIME.strftime('%Y-%m-%d %H:%M:%S')}")
    first_run = True

    while True:
        # ✅ static_only 모드에 따라 분기
        static_sites = [s for s in TARGET_SITES if not s.get('dynamic')]
        dynamic_sites = [] if static_only else [s for s in TARGET_SITES if s.get('dynamic')]

        def fetch_articles(site):
            return (site, get_latest_articles(site))

        with ThreadPoolExecutor(max_workers=6) as executor:
            static_results = executor.map(fetch_articles, static_sites)

        results = list(static_results)

        if not static_only:
            results += [(site, get_latest_articles(site, driver)) for site in dynamic_sites]

        for site, article_list in results:
            site_key = f"{site['name']} - {site.get('category', '')}"
            if not article_list:
                continue

            # 무조건 article_list[0]만 기준 기사로 사용
            top_article = article_list[0]
            top_url = top_article['url']
            top_title = top_article['title']

            # 첫 루프는 기준만 설정
            if first_run:
                last_seen_urls[site_key] = top_url
                print(f"  🔹 [{site_key}] 초기 기준 기사 설정: {top_title}")
                continue

            # 이미 본 기사면 건너뜀
            if top_url == last_seen_urls[site_key]:
                continue

            # 제목 유사성으로 중복 판단
            if is_similar_title(top_title):
                continue

            # 세부 정보 파싱
            details = fetch_article_details(top_url, site=site, driver=driver)
            if not details or not details['publish_time']:
                continue

            pub_time = details['publish_time']
            if pub_time < SCRIPT_START_TIME:
                continue

            # DB 저장
            save_article_to_mongodb({
                "publisher": site['name'],
                "category": site.get('category', 'N/A'),
                "title": details['title'],
                "published_date": pub_time,
                "content": details['text'],
                "url": top_url
            })

            # 중복 방지 캐싱
            processed_urls.append(top_url)
            recent_titles.append(top_title)  # ✅ 제목 캐시 추가
            last_seen_urls[site_key] = top_url

            print(f"  ✅ 기준 기사 업데이트: {site_key} → {top_title}")


        first_run = False
        print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 다음 체크까지 {CHECK_INTERVAL_SECONDS}초 대기...")
        time.sleep(CHECK_INTERVAL_SECONDS)

if __name__ == "__main__":
    # ✅ 여기서 모드 설정
    STATIC_MODE = True  # True면 정적 사이트만, False면 전부

    driver = None
    if not STATIC_MODE:
        driver = setup_driver()

    try:
        monitor_news(driver=driver, static_only=STATIC_MODE)
    except KeyboardInterrupt:
        print("\n종료됨")
    finally:
        if driver:
            driver.quit()
        client.close()





[실시간 뉴스 모니터링 시작]
  기준 시간: 2025-07-24 14:07:20
  🔹 [서울경제 - 증권] 초기 기준 기사 설정: 현대차證, 올 상반기 순익 400억…반년 만에 지난해 연간 실적 '추월'
  🔹 [서울경제 - 경제] 초기 기준 기사 설정: 한은, 집중호우 피해 중소기업 지원…금융중개지원대출 300억 배정
  🔹 [서울경제 - 산업] 초기 기준 기사 설정: 李대통령, 오늘 이재용 만날 듯…재계와 연쇄 회동
  🔹 [서울경제 - 국제] 초기 기준 기사 설정: 관세청, 원산지표시위반 일제점검…671억상당 위반행위 적발
  🔹 [매일경제 - 경제] 초기 기준 기사 설정: 퍼시스그룹, 서울시 어린이병원에 1억원 상당 가구 기부대기·휴게공간에 퍼시스 소파 치료공간에 일룸 키즈 가구 배치퍼시스그룹이 어린이 치료환경 개선을 위해 서울시 어린이병원에 1억원 상당의 가구를 기부했다고 24일 밝혔다. 퍼시스그룹은 지난 10일 병원에 현물을 기부하는 방식으로 가구를 전달했다. 이번 기부는 어린이 환자들과 보호자들이 편안하고 안정된 환경에서 치료에 집중할 수 있도록 지원하기 위해 진행됐다.2025-07-24 14:03:1407.242025
  🔹 [매일경제 - 기업] 초기 기준 기사 설정: 퍼시스그룹, 서울시 어린이병원에 1억원 상당 가구 기부대기·휴게공간에 퍼시스 소파 치료공간에 일룸 키즈 가구 배치퍼시스그룹이 어린이 치료환경 개선을 위해 서울시 어린이병원에 1억원 상당의 가구를 기부했다고 24일 밝혔다. 퍼시스그룹은 지난 10일 병원에 현물을 기부하는 방식으로 가구를 전달했다. 이번 기부는 어린이 환자들과 보호자들이 편안하고 안정된 환경에서 치료에 집중할 수 있도록 지원하기 위해 진행됐다.2025-07-24 14:03:1407.242025
  🔹 [매일경제 - 국제] 초기 기준 기사 설정: ‘돈다발’ 들고 도망친 하마스 지도자 아내, 남편 죽었다는 소식에 한 일가자지구 전쟁을 촉발한 팔레스타인 무장정파 하마스 테러의 설계자인 야히야 신와르의 부인이 튀르키예에

### mark 2 kafka없이 돌아가는 버전


In [ ]:
import requests
from bs4 import BeautifulSoup
import newspaper
import time
import difflib
from datetime import datetime, timezone, timedelta
from urllib.parse import urljoin, urlparse, urlunparse
from news_list import TARGET_SITES, CHECK_INTERVAL_SECONDS, MONGO_URI, MONGO_DB_NAME, MONGO_COLLECTION_NAME, STOCK_CODE
from pymongo import MongoClient
from collections import deque
from concurrent.futures import ThreadPoolExecutor, as_completed
from identify_company_module import identify_company  # 실제 모듈명에 맞게 수정

import re
maxlen= 100
# 설정
KST = timezone(timedelta(hours=9))
SCRIPT_START_TIME = datetime.utcnow().replace(tzinfo=timezone.utc).astimezone(KST)
recent_titles = deque(maxlen=maxlen)
processed_urls = deque(maxlen=maxlen)

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Referer': 'https://dealsite.co.kr/',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ko,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive'

})


last_seen_urls = {
    f"{site['name']} - {site.get('category', '')}": None for site in TARGET_SITES
}

client = MongoClient(MONGO_URI)
db = client[MONGO_DB_NAME]
articles_collection = db[MONGO_COLLECTION_NAME]

# def is_similar_title(new_title, threshold=0.9):
#     for past_title in recent_titles:
#         similarity = difflib.SequenceMatcher(None, new_title, past_title).ratio()
#         if similarity >= threshold:
#             print(f"⚠️ 제목 유사도 중복 판단 →\n    ▸ 새로운 제목: {new_title}\n    ▸ 유사한 제목: {past_title}\n    ▸ 유사도: {similarity:.3f}")
#             return True
#     return False

def localize_to_kst(dt):
    if dt is None:
        return None
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc).astimezone(KST)
    return dt.astimezone(KST)

def clean_url(url):
    parsed = urlparse(url)
    return urlunparse(parsed._replace(query=""))

def fetch_article_details(url, site=None):
    try:
        #url = clean_url(url)
        article = newspaper.Article(url, language='ko')
        article.download()
        article.parse()
        publish_time = localize_to_kst(article.publish_date)

        return {
            'title': article.title,
            'text': article.text,
            'publish_time': publish_time,
            'url': url
        }

    except Exception as e:
        print(f"[ERROR] 기사 처리 중 오류 (건너뜀): {url}\n  └ {e}")
        return None

def save_article_to_mongodb(article_data):
    try:
        articles_collection.insert_one(article_data)
        print(f"  [DB 저장] ({article_data['publisher']}) {article_data['title']}")
    except Exception as e:
        print(f"[ERROR] DB 저장 실패: {e}")

def get_latest_articles(site):
    try:
        if not site.get('selector'):
            print(f"[ERROR] {site['name']} - {site.get('category', '')} → selector 누락")
            return []

        res = session.get(site['url'], timeout=10)
        if res.status_code == 403:
            print(f"[ERROR] {site['name']} 접근 거부됨 (403 Forbidden): {site['url']}")
            return []

        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'html.parser')

        if site.get("container_selector"):
            container = soup.select_one(site["container_selector"])
            if not container:
                print(f"[ERROR] {site['name']} → container_selector 영역을 찾을 수 없습니다.")
                return []
            links = container.select(site["selector"])
        else:
            links = soup.select(site["selector"])

        articles = []
        for a in links:
            href = a.get('href')
            if not href or site['pattern'] not in href:
                continue

            # 언론사별로 제목 태그 분기 처리
            if site['name'] in ["매일경제", "헤럴드경제", "한겨레"]:
                if site['name'] == "매일경제":
                    title_tag = a.select_one("h3.news_ttl")
                elif site['name'] == "헤럴드경제":
                    title_tag = a.select_one("p.news_title")
                elif site['name'] == "한겨레":
                    title_tag = a.select_one("div.BaseArticleCard_title__TVFqt")
                if not title_tag:
                    continue
                title = title_tag.get_text(strip=True)
            else:
                title = a.get_text(strip=True)

            full_url = urljoin(site['base_url'], href)
            articles.append({'title': title, 'url': full_url})


        return articles
    except Exception as e:
        print(f"[ERROR] 목록 수집 실패 ({site['name']}) → {e}")
        return []

def monitor_news():
    print("\n[실시간 뉴스 모니터링 시작]")
    print(f"  기준 시간: {SCRIPT_START_TIME.strftime('%Y-%m-%d %H:%M:%S')}")
    first_run = True

    while True:
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(get_latest_articles, site): site for site in TARGET_SITES if not site.get('dynamic')}

            for future in as_completed(futures):
                site = futures[future]
                try:
                    article_list = future.result()
                except Exception as e:
                    print(f"[ERROR] {site['name']} 처리 실패 → {e}")
                    continue

                site_key = f"{site['name']} - {site.get('category', '')}"
                if not article_list:
                    continue

                top_article = article_list[0]
                top_url = top_article['url']
                top_title = top_article['title']

                if first_run:
                    last_seen_urls[site_key] = top_url
                    print(f"  🔹 [{site_key}] 초기 기준 기사 설정: {top_title}")
                    continue

                # 1. 가장 상단 뉴스 URL이 이전과 동일하다면 skip
                if top_url == last_seen_urls[site_key]:
                    continue

                # # 2. 유사 제목 필터링
                # if is_similar_title(top_title):
                #     continue

                # 3. 뉴스 상세 정보 가져오기
                details = fetch_article_details(top_url, site=site)
                if not details or not details['publish_time']:
                    continue

                pub_time = localize_to_kst(details['publish_time'])
                if pub_time < SCRIPT_START_TIME:
                    continue

                if top_title in recent_titles or top_url in processed_urls:
                    continue

                recent_titles.append(top_title)
                processed_urls.append(top_url)

                # ✅ 관련 기업 태깅
                company_name, max_score, _ = identify_company(details['title'],details['text'])
                if company_name == "미국" or company_name == "코스피" or company_name == "다른기업":
                    company_code = None
                else:
                    company_code = STOCK_CODE[company_name]

                # ✅ DB 저장
                save_article_to_mongodb({
                    "publisher": site['name'],
                    "category": site.get('category', 'N/A'),
                    "title": details['title'],
                    "published_date": pub_time,
                    "content": details['text'],
                    "url": top_url,
                    "company_tag": company_name,
                    "company_score": max_score,
                    "stock_code": company_code
                })


                last_seen_urls[site_key] = top_url  # ✅ 저장 성공 후에만 실행


                print(f"  ✅ 기준 기사 업데이트: {site_key} → {top_title}")

        first_run = False
        print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 다음 체크까지 {CHECK_INTERVAL_SECONDS}초 대기...")
        time.sleep(CHECK_INTERVAL_SECONDS)

if __name__ == "__main__":
    try:
        monitor_news()
    except KeyboardInterrupt:
        print("\n종료됨")
    finally:
        client.close()

# 테스트 코드


In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

headers = {
    'User-Agent': 'Mozilla/5.0'
}

def test_static_site(site_config):
    try:
        print(f"\n[테스트 시작] {site_config['name']} - {site_config.get('category', '')}")
        res = requests.get(site_config['url'], headers=headers, timeout=10)
        res.raise_for_status()

        soup = BeautifulSoup(res.text, 'html.parser')

        links = soup.select(site_config["selector"])
        found_articles = []

        for a in links:
            href = a.get('href')
            title = a.get_text(strip=True)
            if not href or site_config['pattern'] not in href:
                continue
            full_url = urljoin(site_config['base_url'], href)
            found_articles.append({'title': title, 'url': full_url})

        if not found_articles:
            print("❌ 실패: 기사 리스트를 찾지 못했습니다.")
        else:
            print(f"✅ 성공: {len(found_articles)}개의 기사 발견")
            for article in found_articles[:3]:
                print(f"  - {article['title']}")
                print(f"    {article['url']}")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")

if __name__ == "__main__":
    test_config = {
    "name": "뉴시스 경제 최신",
    "category": "경제",
    "url": "https://www.newsis.com/economy/list/?cid=10400&scid=10401",  # 실제 사용하는 페이지 URL로 수정 가능
    "base_url": "https://www.newsis.com",
    "selector": "ul.articleList2 li p.tit a",  # 제목과 링크가 있는 <a> 태그
    "pattern": "/view/NISX",                   # 기사 본문 URL 패턴
    "dynamic": False
}








    test_static_site(test_config)



[테스트 시작] 뉴시스 경제 최신 - 경제
✅ 성공: 20개의 기사 발견
  - 분뇨에서 에너지로…정부, 가축분 고체연료 상업화 시동
    https://www.newsis.com/view/NISX20250724_0003265117
  - 행안장관 "소비쿠폰, 취약계층 노출 없도록"…현장 점검도(종합)
    https://www.newsis.com/view/NISX20250724_0003264976
  - 두산건설 위브 골프단, 26일 부산·인천서 재능 나눔행사
    https://www.newsis.com/view/NISX20250724_0003265087


In [4]:
import requests
from bs4 import BeautifulSoup

url = "https://www.fnnews.com/section/002001002002"

res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
print(soup.prettify()[:3000])  # HTML 일부 출력


<!DOCTYPE html>
<html lang="ko-KR">
 <head>
  <meta charset="utf-8"/>
  <meta content="website" property="og:type"/>
  <meta content="ko_KR" property="og:locale"/>
  <meta content="https://www.fnnews.com/section/002001002002" property="og:url"/>
  <meta content="https://www.fnnews.com/section/002001002002" property="twitter:url"/>
  <meta content="summary" property="twitter:card"/>
  <meta content="_PCSAF7iS_tRVsJyfZCzU-cBZlzdjYJMF6Zfc1LAPJ4" name="google-site-verification"/>
  <meta content="57689B7978DDFA5E64650156AB2E9050" name="msvalidate.01"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="ko" http-equiv="Content-Language"/>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="user-scalable=yes, initial-scale=1.0, minimum-scale=1.0,maximum-scale=5, width=device-width, target-densitydpi=medium-dpi" name="viewport"/>
  <link href="https://www.fnnews.com/section/002001002002" rel="canonical"/>
  <!-- Chrome, Safari, IE -->



[실시간 뉴스 모니터링 시작]
  기준 시간: 2025-07-28 21:32:35
  Kafka 설정: localhost:9092
  🔹 [매일경제 - 경제] 초기 기준 기사 설정: 서울역까지 20분대…지금 그곳은 축제
  🔹 [매일경제 - 기업] 초기 기준 기사 설정: “테슬라 브랜드 파워 부럽다고?...기업은 적자에도 팬덤부터 관리해야”
  🔹 [매일경제 - 금융] 초기 기준 기사 설정: 계속되는 알트코인 순환매…이번엔 바이낸스코인 천장 뚫었다
  🔹 [매일경제 - 국제] 초기 기준 기사 설정: 미국 기업대표단 이번주 중국행…G2 관세전쟁 결판짓나
  🔹 [서울경제 - 증권] 초기 기준 기사 설정: 채비·씨엠티엑스·알지노믹스 상장 예심 청구 [시그널]
  🔹 [서울경제 - 국제] 초기 기준 기사 설정: [속보] 구윤철, 美베선트와 협상차 내일 출국
  🔹 [서울경제 - 산업] 초기 기준 기사 설정: 노조법 개정안 환노위 통과…경총 "산업 생태계 붕괴, 강행 처리 중단하라"
  🔹 [서울경제 - 경제] 초기 기준 기사 설정: 'MASGA' 띄운 韓…김동관도 美서 합류
  🔹 [연합뉴스 - 경제] 초기 기준 기사 설정: 李대통령, 러트닉 따라간 김정관·여한구에 "끝까지 최선" 당부(종합)
  🔹 [연합뉴스 - 국제] 초기 기준 기사 설정: 튀르키예 산불 강풍 타고 사흘째 확산…4명 사망
  🔹 [연합뉴스 - 금융] 초기 기준 기사 설정: 국고채 금리, 한미 관세협상 등 주시 속 하락…3년물 2.464%(종합)
  🔹 [연합뉴스 - 산업] 초기 기준 기사 설정: [K-VIBE] 이은준의 AI 톺아보기…AI 영상 시대, 감독의 역할
  🔹 [이투데이 - 경제] 초기 기준 기사 설정: 구윤철 부총리, 31일 美재무장관과 '막판 협상'..."통상 협상에 총력 대응"
  🔹 [이투데이 - 산업] 초기 기준 기사 설정: 네이버클라우드ㆍNHN클라우드ㆍ카카오, 정부 GPU 사업자로 선정
  🔹 [한겨레 - 경제] 초기 기준 기사 설정: 정부, 수십조 조선 협력 ‘마스가’ 

# DB 체크

In [ ]:
from pymongo import MongoClient
import pandas as pd

MONGO_URI = 'mongodb://localhost:27017'
MONGO_DB_NAME = 'stock'
MONGO_COLLECTION_NAME = 'new_news'

client = MongoClient(MONGO_URI)
db = client[MONGO_DB_NAME]
articles_collection = db[MONGO_COLLECTION_NAME]

data = list(articles_collection.find())  # find()는 기본적으로 cursor를 반환
df = pd.DataFrame(data)


,_id,publisher,category,title,published_date,content,url,company_tag
0,6881f6da0291f6db97e44d05,한국경제,경제,관세비용 8200억…美서 더 팔고도 영업익 꺾인 현대차,2025-07-24 09:01:50,2분기 '美 최대판매' 찍었지만…영업이익 15.8% 감소\n\n\n\n관세 없었다면...,https://www.hankyung.com/article/2025072404571,NaN
1,6881f6f80291f6db97e44d06,한국경제,국제,"태국·캄보디아, 국경서 중화기로 교전",2025-07-24 09:02:36,국경 분쟁 문제로 갈등을 빚고 있는 태국과 캄보디아가 국경 지역에서 다연장로켓포 등...,https://www.hankyung.com/article/2025072404291,NaN
2,6881f6ff0291f6db97e44d07,서울경제,국제,서울경제신문 윤경환 신임 뉴욕특파원 부임,2025-07-24 09:03:55,"< 저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지 >\n\n윤경환(사진) 서울경...",https://www.sedaily.com/NewsView/2GVH4O1N1A/GF02,NaN
3,6881f7090291f6db97e44d08,연합뉴스,산업,고준위방폐장 연구시설 선정 놓고 학계-원자력환경공단 공방,2025-07-24 09:01:50,"손대성기자 구독 구독중 이전 다음\n\n학계 ""태백 부지 퇴적암층이어서 부적절"", ...",https://www.yna.co.kr/view/AKR2025072416710005...,NaN
4,6881f7180291f6db97e44d09,한국경제,경제,"""기후협약 위반 국가 손해배상 물을 수도""",2025-07-24 09:02:58,"국제사법재판소 ""국제법 위반""\n\n국제사법재판소(ICJ)가 기후변화협약을 이행하지...",https://www.hankyung.com/article/2025072404301,NaN
...,...,...,...,...,...,...,...,...
3394,68872b1f29f6fc66688a65af,머니투데이,증권,"1억→3.5억…파죽지세 한화그룹株, 지금이라도 '승선'할까?",2025-07-28 07:46:30,올해 방산·조선 랠리에 힘입어 한화그룹주가 고공행진을 이어가고 있다. 증권가에서는 ...,https://news.mt.co.kr/mtview.php?no=2025072812...,다른기업
3395,68872b4229f6fc66688a65b0,한국경제,국제,"손흥민 떠나면 팬들도 간다?…""토트넘 매년 1100억 손실""",2025-07-28 07:46:46,손흥민. 사진=뉴스1\n\n손흥민(33)이 잉글랜드 프리미어리그(EPL) 토트넘 홋...,https://www.hankyung.com/article/2025072868247,다른기업
3396,68872b7129f6fc66688a65b1,머니투데이,산업,"울트라브이, 배우 한채아와 함께한 울트라콜 신규 캠페인 선봬",2025-07-28 07:49:00,\n\n\n\n사진제공=울트라브이 이번 캠페인의 메인 슬로건인 '이제는 울트라콜' ...,https://news.mt.co.kr/mtview.php?no=2025072811...,다른기업
3397,68872b9f29f6fc66688a65b2,뉴시스,경제,[올댓차이나]중국 증시 상승 마감…상하이지수 0.12%↑,2025-07-28 07:49:12,"1예멘 후티군, 이스라엘과 관련된 ""모든 외국 선박들""에 공격 예고",https://www.newsis.com/view/NISX20250728_00032...,다른기업


In [10]:
df = df.dropna(subset=["company_tag"])
df[df['company_tag'] == '카카오'].to_csv("kakao_news.csv", index = False, encoding='UTF-8-sig')


In [13]:
import pandas as pd
stock_code = pd.read_csv("상장기업98.csv", encoding='utf-8-sig')
select_columns = ['corp_name', 'stock_code']
stock_code = stock_code[select_columns]
corp_dict = dict(zip(stock_code['corp_name'], stock_code['stock_code']))
print(corp_dict)    

{'삼성전자': 5930, 'SK하이닉스': 660, 'LG에너지솔루션': 373220, '삼성바이오로직스': 207940, '현대자동차': 5380, '기아': 270, '셀트리온': 68270, '한화에어로스페이스': 12450, 'KB금융': 105560, 'NAVER': 35420, 'HD현대중공업': 329180, '신한지주': 55550, '현대모비스': 12330, '한화오션': 42660, '메리츠금융지주': 138040, 'POSCO홀딩스': 5490, '삼성물산': 28260, '크래프톤': 259960, '카카오': 35720, 'HMM': 11200, '삼성화재': 810, 'LG화학': 51910, '하나금융지주': 86790, '삼성생명': 32830, 'SK이노베이션': 96770, 'HD한국조선해양': 9540, '한국전력': 15760, '고려아연': 10130, '두산에너빌리티': 34020, 'KT&G': 33780, '삼성중공업': 10140, '삼성SDI': 6400, 'KT': 30200, 'SK텔레콤': 17670, '우리금융지주': 316140, 'SK스퀘어': 402340, '기업은행': 24110, 'HD현대일렉트릭': 267260, 'LG전자': 66570, '현대로템': 64350, '포스코퓨처엠': 3670, '카카오뱅크': 323410, 'LG': 3550, '하이브': 352820, '포스코인터내셔널': 47050, '삼성전기': 9150, 'SK': 34730, '삼성에스디에스': 18260, '유한양행': 100, '현대글로비스': 86280, '대한항공': 3490, 'SK바이오팜': 326030, '한국항공우주': 47810, 'HD현대마린솔루션': 443060, '삼양식품': 3230, '한화시스템': 272210, '한미반도체': 42700, '아모레퍼시픽': 90430, 'DB손해보험': 5830, 'S-Oil': 10950, 'LIG넥스원': 79550, 'HD현대': 267250, '한진칼